# Lab 4:  Regularization

## Saturday, November 3rd 2018

#### David Sondak and Pavlos Protopapas

# Background
Lecture 3 introduced several types of regularization.  In today's lab, you will become more familiar with those regularization techniques and actually apply them to a problem.  The types of regularization that you will explore today are:
* Penalization
* Early stopping
* Dropout
There are many other types of regularization (as mentioned in lecture).  The three regularization techniques that you will explore today are very popular and used frequently in real applications.

We'll begin the story by building a neural network to learn a function from some noisy data.

# Warming Up
Today we'll try to fit the function $$f\left(x\right) = x\sin\left(x\right).$$

Using `keras`, build a fully-connected neural network to fit $f\left(x\right)$.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import models
from keras import layers
%matplotlib inline

First, we'll generate some synthetic data with some synthetic noise.

In [ ]:
n_samples = 100 # set the number of samples to take for each toy dataset
test_size = 0.2 # set the proportion of toy data to hold out for testing
random_seed = 1 # set the random seed to make the experiment reproducible 
np.random.seed(random_seed)

# define a function
f = lambda x: x * np.sin(x)

# Generate the truth function (without any noise)
X_true = np.linspace(0.0, 5.0, n_samples)
Y_true = f(X_true)

# Now sample the true function at some points
X = np.random.permutation(X_true) # choose some points from the function - this is our toy dataset 
Y = f(X)

Y = Y + np.random.normal(0.0, 1.0, len(Y)) # Add some noise from a random normal distribution

# create training and testing data from this set of points
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)

Now we'll build a network.  We choose $5$ hidden layers and $100$ nodes per layer.

In [ ]:
num_layers = 5
N = 100

input_dim = 1

model = models.Sequential()

model.add(layers.Dense(N, input_dim=input_dim, activation='relu'))

# Create hidden layers
for h in range(num_layers):
    model.add(layers.Dense(N, activation='relu'))
    
model.add(layers.Dense(1, activation='linear'))

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

Next we'll fit the model.  

Notice that we're specifying a *validation set*.  What this means is that `keras` will further split the training set into a training part and a validation part.  The neural network will only be trained on the *training* set.  Meanwhile, `keras` will report performance metrics on the *validation* set so we can get a sense of how well the model has been trained.  We will be using the validation set quite a bit in this lab.

Remember, we don't want to use the test set for anything relating to the training of our models.  By withholding the validation set, we can assess the model performance on the validation set.  Later, we can see how the model performs on data it has never seen before by using in on the test set.

In [ ]:
# Fit the model
no_reg = model.fit(X_train, Y_train, epochs=2500, batch_size=64, validation_split=0.2)

Now the model is trained.  Let's see what the solution looks like.

We will also get the validation data set that `keras` created for us.  By plotting the validation data, we will gain some insight into how well the model generalizes.

In [ ]:
# Validation set
X_val = no_reg.validation_data[0]
Y_val = no_reg.validation_data[1]

In [ ]:
# use our model to predict in the range we want
X_range = np.linspace(0.0, 5, 1000)
y_pred = model.predict(X_range)

# Now plot everything
fig, ax = plt.subplots(1, 1, figsize=(14,8))
ax.plot(X_true, Y_true, ls='-.', lw=4, label='True function')
ax.plot(X_train, Y_train, '.', label='Training data')
ax.plot(X_val, Y_val, '.', label='Validation data')
ax.plot(X_range, y_pred, lw=4, label='Prediction')

ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.tick_params(labelsize=20)

ax.legend(loc=1, fontsize=20)

plt.show()

The prediction looks pretty bad.  The neural network model is trying to go through all the training points.  This is a classic case of overfitting.  The solution has a lot of oscillations and it rarely fits the validation data.  It may be a good idea to use some kind of regularization.

Let's begin with some penalization methods.

# Penalization
As mentioned in lecture, the loss function can be augmented by an additional term called the penalization term.  Mathematicall, the goal is to find the set of weights $W$ that minimize the functional $$J_{R}\left(W; X, y\right) = J\left(W; X, y\right) + \alpha\Omega\left(W\right)$$
where $\alpha$ is called the regularization (or penalization) parameter.  In this lab, $\displaystyle J\left(W; X, y\right)$ is the MSE loss function.

Next, we consider the effect of two different forms for the penalization term: $L_{1}$ and $L_{2}$ penalization.  For reference, 
$$\Omega_{L_{1}} = \frac{1}{2}\left\|W\right\|_{1}$$
and 
$$\Omega_{L_{2}} = \frac{1}{2}\|W\|^{2}_{2}.$$

Note that the biases are not being penalized.

## Exercise
Fit the same network as above ($5$ hidden layers, $100$ nodes per layer, linear output layer), but this time use $L_{2}$ and $L_{1}$ regularization.

**Deliverables:**
* Make two figures, one on top of the other.
* The first figure should contain the following:
  - True solution
  - Training data
  - Validation data
  - Neural network prediction without regularization
  - Neural network prediction with $L_{2}$ regularization
* The second figure should contain the following:
  - True solution
  - Training data
  - Validation data
  - Neural network prediction without regularization
  - Neural network prediction with $L_{1}$ regularization
* **Make sure everything is clearly labeled!**
* Discuss the results.

**Hints:**
* Use `kernel_regularizer=regularizers.l2(alpha)` after the `activation` argument in each of your layers.
* Choose a value for `alpha` that you think works well.  You may need to play around with this a little bit.
* See the `Keras` documentation on regularization:  [Usage of regularizers](https://keras.io/regularizers/)
* Here's some pseudo-code:

```python
from keras import regularizers

num_layers = 5
N = 100
alpha = 

input_dim = 1

### Create network
model_L2 = 
model_L2.add()


### Compile network
model_L2.compile()

### Fit model
L2_reg = model_L2.fit()

### Extract validation data
X_val_L2 = 
Y_val_L2 = 

### REPEAT FOR L1
###
###
###

# PLOT
fig, ax = plt.subplots(2, 1, figsize=(20,14), sharex=True)

ax[0].plot() # Top plots
### ...

ax[0].set_ylabel(r'$Y$', fontsize=20)
ax[0].tick_params(labelsize=20)
ax[0].legend(loc=1, fontsize=20)


ax[1].plot() # Bottom plots
### ...
ax[1].set_xlabel(r'$Y$', fontsize=20)
ax[1].set_ylabel(r'$Y$', fontsize=20)
ax[1].tick_params(labelsize=20)
ax[1].legend(loc=1, fontsize=20)

plt.show()
```

### Solution

# Early Stopping
The results without any regularization do not look right.  $L_{2}$ and $L_{1}$ regularizaton helped somewhat, but the results still aren't convincing.

We can gain some more insight by plotting the loss functions from the training and validation set.  Let's use a `log-log` scale to enhance any discrepancies between the two curves.

First, a reminder.

Remember that the `fit` method can store the history of the model.  For the unregularized model we stored all the history in the name `no_reg`.  Let's see what attributes are in that object.

In [ ]:
dir(no_reg)

There is a lot of stuff; most of it we're not interested in.  However, at the very end of the list, we see some useful keys.  Let's access some of them.

In [ ]:
type(no_reg.history)

Looks like `history` is a dictionary.  Let's take a look at it's keys.

In [ ]:
no_reg.history.keys()

Very cool.  There is a `validation` and `training` loss.  

That's the one we'll want to use right now, but we can look at the other attributes too just to get a feel.

In [ ]:
type(no_reg.validation_data)

In [ ]:
type(no_reg.params)

In [ ]:
no_reg.params.keys()

In [ ]:
no_reg.params['batch_size']

Okay, that was fun and informative.  But what we're really after is the loss data as a function of epoch number.  Here we go.

In [ ]:
L = no_reg.history['loss']
L_val = no_reg.history['val_loss']

fig, ax = plt.subplots(1,1, figsize=(10,6))

ax.plot(L, label='Training Loss')
ax.plot(L_val, label='Validation Loss')

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel(r'Epochs', fontsize=20)
ax.set_ylabel(r'Loss', fontsize=20)
ax.tick_params(labelsize=20)

ax.legend(loc=1, fontsize=20)

plt.show()

Wow.  That is striking.

We used $2500$ epochs, but the validation loss begins to rise at around $50$ epochs and becomes larger than the training loss at around $70$ epochs.  After that, we're basically overfitting.

Notice that the training loss keeps decreasing.  We're fitting the training data better and better all the time.  The validation loss is getting larger and larger meaning that we're losing generalizability.

We can use this new information to our advantage!

## Exercise

### Part 1
Train a network without any penalization, but this time stop after $20$ epochs.

### Part 2
Train a network without any penalization, but this time stop at the "optimal" number of epochs (based on the crossing of the loss curves).

**Deliverables**
* Plot the following on a single figure:
  - The true solution
  - The model prediction without any regularization (after $2500$ epochs)
  - The model prediction without any regularization using $20$ epochs
  - The model prediction without any regularization using the optimal number of epochs
* You may also want to include the training and validation data on the same plot.  Be careful that the plot doesn't become too cluttered.

### Solution

Let's try to do things more systematically.

How do you think early stopping should be implemented?

To do early stopping in `Keras`, you specify the `EarlyStopping` [*callback*](https://keras.io/callbacks/).  From the documentation:
> A callback is a set of functions to be applied at given stages of the training procedure.

Callbacks can be used to view internal states and statistic of the model during training.

Right now, we'll use one to monitor the validation loss function.  When the validation loss starts to go up, the training process will stop.

#### Basic Syntax
To specify a callback, you just pass a `callbacks` list into the model `fit()` method, like this:
```python
from keras.callbacks import EarlyStopping

model.fit(X_train, Y_train, epochs=2500, batch_size=64, validation_split=0.2, 
          callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
```

## Exercise
Fit the model using the early stopping technique.  Try different values for `patience` to see which one gives you the lowest validation loss.

How many epochs are needed?

### Solution

The solution should looks pretty good.  Of course, we had to play with the `patience` parameter.

# Dropout
The basic idea behind the dropout technique is to randomly remove (i.e drop) at unit from the network including all of its connections.  Why would we want to do such a thing?  Because we want to avoid overfitting.  An intuitive way of seeing this is that by randomly dropping units at each iteration, we are actually training an ensemble of models.  This ensemble may be more robust to different scenarios than a single model.

One usually needs to figure out how many units to drop in each layer.  Of course, this introduces a new hyperparameter to tune.  However, at least empirically, it appears that choosing a dropout rate of $0.5$ in the hidden layers and $0.8$ in the input layers works well.  Note that here the term "dropout rate" means "the probability of retention of each node in the layer".

Dropout usually works best on large networks.  We will use it on our small lab network today just to understand some of the mechanics.

The original paper on dropout has some nice explanations of the technique and also some really nice practical advice on using the technique:  [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf).

In [ ]:
from keras.constraints import maxnorm

num_layers = 5
N = 100

input_dim = 1

model = models.Sequential()

model.add(layers.Dense(N, input_dim=input_dim, activation='relu'))

# Create hidden layers
for h in range(num_layers):
    model.add(layers.Dense(N, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(layers.Dropout(0.5))

model.add(layers.Dense(1, activation='linear'))

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit model
no_reg_dropout = model.fit(X_train, Y_train, epochs=250, batch_size=64, validation_split=0.2)